In [ ]:
import pandas as pd
import os
from google.colab import drive
import shutil
from sklearn.model_selection import train_test_split
import reorganize_extracted_data_api
from reorganize_extracted_data_api import *

# This code was used to reorganize the data in our Google Drive

# Original Organization Format

After extracting all the audio files, they were in the following format:

Extracted_data (root)

├── 20200505 (dir giving date)

│   ├── 0Ha52POVIxTKEPqI1eGpIoMHUd52 (dir giving ID)

│   │   └── breathing-deep.wav (audio file)

│   │   └── breathing-shallow.wav (audio file)

│   │   └── cough-heavy.wav (audio file)

│   │   └── *********.wav etc. (audio files)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3 (dir giving ID)

│   │   └── cough-heavy.wav (audio file)

│   │   └── *********.wav etc. (audio files)

│   ├── ************************* etc. (dir giving ID)

├── 20200525 (dir)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3 (dir giving ID)

│   │   └── cough-heavy.wav (audio file)

│   │   └── *********.wav etc. (audio files)

│   ├── ************************* etc. (dir giving ID)

└── ######## etc. (dir)

# New Organization Format

This code then reorganizes our files into separate folders, one for each of the sounds. The only contents of these folders are the audio files which have been edited to include the ID in their names

Extracted_data (root)

├──  breathing-deep (dir giving type of audio)

│   ├── 0Ha52POVIxTKEPqI1eGpIoMHUd52_breathing-deep.wav (audio file w/ID)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3_breathing-deep.wav (audio file w/ID)

│   │   └── *********_******.wav etc. (audio files w/IDs)

├──  breathing-shallow (dir giving type of audio)

│   ├── 0Ha52POVIxTKEPqI1eGpIoMHUd52_breathing-shallow.wav (audio file w/ID)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3_breathing-shallow.wav (audio file w/ID)

│   │   └── *********_******.wav etc. (audio files w/IDs)

├──  ***** etc. (dir giving type of audio)

│   │   └── *********_******.wav etc. (audio files w/IDs)


In [ ]:
# Mount Google Drive
drive.mount("/content/gdrive", force_remount=True) # Run and choose your school google drive and copy and paste the key and press enter

Mounted at /content/gdrive


In [ ]:
# Set path you want to clean up and path to move to
path = '/content/gdrive/MyDrive/DSCI400/Coswara-Data-master/Coswara-Data-master/Extracted_data'
new_path = '/content/gdrive/MyDrive/DSCI400/Reorganized Audio 2'

In [ ]:
# Create new directory
make_new_dir(new_path)

# Set working directory
os.chdir(path) 

desired_directories = ['breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow', 'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o', 'metadata']

In [ ]:
# The first function call will simply remove the first layer of directories
#   beneath the root folder as defined in the path variable. It does not edit
#   any of the names or contents of the subfolders.
cleaner(path, new_path, desired_directories, 0)

In [ ]:
# The second function call will rename the files in each remaining subfolder
#   in the root folder to include the name of the subfolder, followed by an
#   underscore, followed by the original name of the file. It then sorts the
#   renamed file into a folder which only contains files of its corresponding
#   sound. The "1" is used as a flag to represent this case.
cleaner(new_path, new_path, desired_directories, 1)

In [ ]:
# Change to the new path we want to move our files to
os.chdir(new_path)

# Organizes the files in the source directory into their appropriate 
#   subdirectories within the source directory

org_into_sounds(new_path, desired_directories)